In [1]:
# Install necessary packages
!pip install tqdm torch torchvision pytorch-msssim

import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

from pathlib import Path
import random
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from tqdm import tqdm
from PIL import Image
import torch.nn.functional as f
from pytorch_msssim import ssim
from models.autoencoder.auto import Auto  # Import your custom Auto class

# ✅ Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ Paths
DATA_DIR = Path("../data/SD1")
PROCESSED_TRAIN_DIR = DATA_DIR / "processed_train"
PROCESSED_VAL_DIR = DATA_DIR / "processed_val"
MODEL_DIR = Path("../models")
MODEL_DIR.mkdir(parents=True, exist_ok=True)  # Ensure model directory exists


# ✅ Dataset class
class GlareRemovalDataset(Dataset):
    def __init__(self, root_dir: str, _transform: transforms.Compose = None, limit: int = None):
        self.root_dir = Path(root_dir)
        self.transform = _transform
        self.image_pairs = []

        # Pair `glare_xxx.png` with `gt_xxx.png`
        for glare_img in sorted(self.root_dir.glob("glare_*.png")):
            gt_img = self.root_dir / glare_img.name.replace("glare_", "gt_")
            if gt_img.exists():
                self.image_pairs.append((glare_img, gt_img))

        # Limit dataset size for debugging
        if limit:
            self.image_pairs = random.sample(self.image_pairs, min(limit, len(self.image_pairs)))

    def __len__(self) -> int:
        return len(self.image_pairs)

    def __getitem__(self, idx: int):
        glare_path, gt_path = self.image_pairs[idx]
        glare_img = Image.open(glare_path).convert("RGB")
        gt_img = Image.open(gt_path).convert("RGB")

        if self.transform:
            glare_img = self.transform(glare_img)
            gt_img = self.transform(gt_img)

        return glare_img, gt_img


# ✅ Transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to tensor (values between 0-1)
])

# ✅ Data loaders (Optimized)
train_dataset = GlareRemovalDataset(PROCESSED_TRAIN_DIR, _transform=transform)
val_dataset = GlareRemovalDataset(PROCESSED_VAL_DIR, _transform=transform)

train_loader = DataLoader(
    train_dataset, batch_size=16, shuffle=True, num_workers=0, pin_memory=True
)
val_loader = DataLoader(
    val_dataset, batch_size=16, shuffle=False, num_workers=0, pin_memory=True
)

# ✅ Instantiate Model
model = Auto().to(device)


# ✅ Custom Loss (L1 + SSIM Loss)
def custom_loss(predicted: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    l1_loss = f.l1_loss(predicted, target)
    ssim_loss = 1 - ssim(predicted, target, data_range=1, size_average=True)
    return l1_loss + 0.5 * ssim_loss  # Adjust SSIM weight as needed


# ✅ Optimizer & Scheduler
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)

# ✅ Training loop
NUM_EPOCHS = 30  # Increased for better learning
PATIENCE = 5
counter = 0
best_val_loss = float('inf')

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0

    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{NUM_EPOCHS}"):
        images, targets = images.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = custom_loss(outputs, targets)
        loss.backward()

        # ✅ Gradient Clipping to avoid exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1} - Training Loss: {avg_train_loss:.4f}")

    # ✅ Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, targets in val_loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            val_loss += custom_loss(outputs, targets).item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch + 1} - Validation Loss: {avg_val_loss:.4f}")

    # ✅ Adjust learning rate if needed
    scheduler.step(avg_val_loss)

    # ✅ Save best model checkpoint
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        counter = 0
        best_model_path = MODEL_DIR / "best_glare_removal_autoencoder.pth"
        torch.save(model.state_dict(), best_model_path)
        print(f"✅ Best Model Saved at: {best_model_path}")
    else:
        counter += 1
        if counter >= PATIENCE:
            print(f"Early stopping triggered after {epoch + 1} epochs")
            break

    # ✅ Periodic checkpoint saving
    if (epoch + 1) % 5 == 0:
        checkpoint_path = MODEL_DIR / f"glare_autoencoder_epoch_{epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_path)
        print(f"📌 Checkpoint saved at: {checkpoint_path}")

# ✅ Save final model
final_model_path = MODEL_DIR / "final_glare_removal_autoencoder.pth"
torch.save(model.state_dict(), final_model_path)
print(f"🎉 Final Model Saved at: {final_model_path}")


Using device: cpu


Epoch 1/30: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 750/750 [21:22<00:00,  1.71s/it]


Epoch 1 - Training Loss: 0.3480
Epoch 1 - Validation Loss: 0.1561
✅ Best Model Saved at: ..\models\best_glare_removal_autoencoder.pth


Epoch 2/30: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 750/750 [12:27<00:00,  1.00it/s]


Epoch 2 - Training Loss: 0.1874
Epoch 2 - Validation Loss: 0.1455
✅ Best Model Saved at: ..\models\best_glare_removal_autoencoder.pth


...

Epoch 11 - Training Loss: 0.1772
Epoch 11 - Validation Loss: 0.1459


Epoch 12/30: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 750/750 [09:59<00:00,  1.25it/s]


Epoch 12 - Training Loss: 0.1772
Epoch 12 - Validation Loss: 0.1406


Epoch 13/30: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 750/750 [10:02<00:00,  1.24it/s]


Epoch 13 - Training Loss: 0.1769
Epoch 13 - Validation Loss: 0.1388
Early stopping triggered after 13 epochs
🎉 Final Model Saved at: ..\models\final_glare_removal_autoencoder.pth
